# Notebook #1


### Name: Tulsi Patel


### Date: 11.11.2021


### PID: 730392259


### Goal: Find average firing rate for Brain Observatory, WT, Female, VISp, Static Gratings (SG) and Drifting Gratings (DG). 

*There are no Female SST VISp and VISl sessions. There are no Female WT/WT VISp and VISl sessions. (Disregard the Jupyter notebooks labeled "Female WT". They are not actually WT/WT. They are hybrids. 

## Protocol

### Starting code. 

In [1]:
import os

import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys.ecephys_session import (
    EcephysSession
)
from allensdk.brain_observatory.ecephys.visualization import plot_mean_waveforms, plot_spike_counts, raster_plot
from allensdk.brain_observatory.visualization import plot_running_speed

# tell pandas to show all columns when we display a DataFrame
pd.set_option("display.max_columns", None)

In [2]:
data_directory = '/Users/tulsipatel/local1/ecephys_cache_dir' 
# must be updated to a valid directory in your filesystem

manifest_path = os.path.join(data_directory, "manifest.json")

In [3]:
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

In [4]:
units = cache.get_units()

In [5]:
sessions = cache.get_session_table()


print('Total number of sessions: ' + str(len(sessions)))

sessions.head()

Total number of sessions: 58


,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
721123822,2019-10-03T00:00:00Z,707296982,brain_observatory_1.1,125.0,M,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,444,2229,6,"[MB, SCig, PPT, NOT, DG, CA1, VISam, nan, LP, ..."
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"


In [6]:
filtered_sessions = sessions[(sessions.sex == 'M') & \
                             (sessions.full_genotype.str.find('wt/wt') > -1) & \
                             (sessions.session_type == 'brain_observatory_1.1') & \
                             (['VISp' in acronyms for acronyms in 
                               sessions.ecephys_structure_acronyms])]

filtered_sessions.head()

,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"
739448407,2019-10-03T00:00:00Z,716813543,brain_observatory_1.1,112.0,M,wt/wt,625,2221,6,"[grey, VISam, nan, VIS, VISp, VISl, VISrl]"
742951821,2019-10-03T00:00:00Z,723627604,brain_observatory_1.1,120.0,M,wt/wt,893,2219,6,"[VISal, nan, grey, VISl, VISrl, VISp, VISpm, VIS]"
743475441,2019-10-03T00:00:00Z,722882755,brain_observatory_1.1,121.0,M,wt/wt,553,2225,6,"[LP, LGd, HPF, DG, CA3, CA1, VISrl, nan, PP, P..."


In [7]:
session_id = filtered_sessions.index.values[1]

session = cache.get_session_data(session_id)

Downloading:   0%|          | 0.00/2.18G [00:00<?, ?B/s]

In [8]:
units = session.units

print('Total number of units: ' + str(len(units)))

units.head()

/Users/tulsipatel/.local/lib/python3.8/site-packages/allensdk/brain_observatory/ecephys/ecephys_project_api/ecephys_project_warehouse_api.py:291: FutureWarning: Conversion of the second argument of issubdtype from `bool` to `np.generic` is deprecated. In future, it will be treated as `np.bool_ == np.dtype(bool).type`.
  pv_is_bool = np.issubdtype(output["p_value_rf"].values[0], np.bool)


Total number of units: 568


,isi_violations,isolation_distance,presence_ratio,L_ratio,silhouette_score,waveform_velocity_below,waveform_PT_ratio,peak_channel_id,waveform_spread,amplitude_cutoff,waveform_repolarization_slope,cumulative_drift,waveform_recovery_slope,nn_miss_rate,firing_rate,nn_hit_rate,max_drift,waveform_duration,local_index_unit,waveform_velocity_above,snr,d_prime,waveform_halfwidth,cluster_id,waveform_amplitude,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
951873075,0.007152,217.235358,0.99,0.008822,0.102778,-0.343384,0.474643,850217391,100.0,0.005241,0.420450,109.34,-0.074171,0.068443,28.521041,0.981333,46.12,0.233501,11,-0.215841,2.372665,3.176851,0.151089,11,94.135470,NaN,400.0,2.920165,2.528816,1.203947,3.781982,0.927133,0.278908,0.031626,0.036298,0.060875,80.000,0.258968,0.111,0.02,15.0,-0.281873,-0.292035,-0.250845,-0.287273,-0.270851,135.0,120.0,0.013332,0.011072,0.011968,0.479271,5.469235e-05,-27.500,4952,0.25,27.190068,28.536524,30.375952,22.075780,31.185267,NaN,0.0555,False,False,False,False,False,0.119373,False,0.009507,0.001270,0.004365,0.028352,0.009290,240,27,757988387,23,8.0,grey,NaN,NaN,NaN,probeA,See electrode locations,29999.970319,1249.998763,True
951873097,0.001013,122.484204,0.99,0.000807,0.050535,-0.206030,0.330639,850217393,80.0,0.000001,0.591829,104.84,-0.086391,0.000000,26.797376,0.999333,64.57,0.412060,13,-0.068677,3.967552,6.785515,0.164824,13,170.762865,NaN,200.0,2.857941,2.415720,0.557588,3.520448,0.991270,0.257158,0.017766,0.008091,0.023106,85.000,0.445727,0.290,0.02,15.0,-0.320775,-0.235060,-0.144444,-0.579909,-0.262002,180.0,0.0,0.002854,0.074976,0.241898,0.035710,6.489135e-03,10.000,4969,0.25,26.866547,21.955271,29.236065,18.960631,28.983773,NaN,0.2055,False,False,False,False,False,0.035932,False,0.003915,0.000754,0.001397,0.026542,0.002959,260,43,757988387,24,8.0,grey,NaN,NaN,NaN,probeA,See electrode locations,29999.970319,1249.998763,True
951873063,0.141647,56.159655,0.99,0.006651,0.164055,-0.703119,0.608431,850217393,140.0,0.017298,0.485948,273.99,-0.116500,0.001177,2.388511,0.936720,49.35,0.219765,10,0.000000,2.573229,4.377841,0.137353,10,106.887495,NaN,5600.0,3.989308,0.956190,1.830000,1.133333,1.517370,0.452706,0.020704,0.411478,0.247698,63.214,1.174016,0.111,0.02,8.0,0.678218,0.269737,0.776145,NaN,0.772541,315.0,120.0,0.022920,0.557782,0.000147,NaN,3.605302e-08,6.429,4939,0.50,2.447853,0.586184,3.859631,0.877198,3.770182,NaN,0.0635,False,False,False,False,False,0.150271,False,0.337184,0.138462,0.066968,0.300025,0.165775,260,43,757988387,24,8.0,grey,NaN,NaN,NaN,probeA,See electrode locations,29999.970319,1249.998763,True
951873182,0.308596,70.046500,0.99,0.023951,0.082816,0.255085,0.741227,850217411,90.0,0.033559,0.452099,346.10,-0.169453,0.012990,1.618139,0.962667,35.54,0.151089,20,0.000000,3.017129,4.617693,0.096147,21,131.342640,NaN,900.0,4.943365,1.626667,4.780000,1.435897,1.598639,0.499537,0.047024,0.118419,0.201685,20.000,0.567178,0.000,0.02,8.0,NaN,-1.000000,-0.891156,0.898438

In [9]:
V1_units = units[(units.ecephys_structure_acronym == 'VISp') &
                (units.isi_violations < 0.1)]

print('Total number of low-contamination V1 units: ' + str(len(V1_units)))


V1_units.head()

Total number of low-contamination V1 units: 27


,isi_violations,isolation_distance,presence_ratio,L_ratio,silhouette_score,waveform_velocity_below,waveform_PT_ratio,peak_channel_id,waveform_spread,amplitude_cutoff,waveform_repolarization_slope,cumulative_drift,waveform_recovery_slope,nn_miss_rate,firing_rate,nn_hit_rate,max_drift,waveform_duration,local_index_unit,waveform_velocity_above,snr,d_prime,waveform_halfwidth,cluster_id,waveform_amplitude,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
951867908,0.011092,89.892752,0.99,0.001330,0.182464,-0.343384,0.227340,850220023,50.0,0.008551,0.200838,102.98,-0.015417,0.000860,4.303205,0.986000,26.02,0.659296,335,-0.343384,2.726828,5.268632,0.192295,356,56.674215,NaN,300.0,0.958559,0.619007,1.940984,0.302381,0.932796,0.540565,0.146865,0.221187,0.063126,46.667,0.838075,0.000,0.32,4.0,NaN,0.367959,0.161491,-0.043478,0.190476,0.0,0.0,NaN,4.984320e-03,0.204969,0.799627,0.459532,13.333,4918,0.0,2.648002,4.236515,9.102307,1.703957,3.938042,NaN,0.1975,False,False,False,False,False,0.681390,False,0.119043,0.064390,0.346145,0.680475,0.142756,2560,27,757988391,255,385.0,VISp,NaN,NaN,NaN,probeC,See electrode locations,29999.998448,1249.999935,True
951868041,0.004394,82.085844,0.99,0.000391,0.127191,-0.618090,0.449737,850220031,60.0,0.000388,0.124251,113.14,-0.030577,0.000246,3.057497,0.996640,41.02,0.686767,344,2.060302,2.577298,5.497135,0.274707,365,60.255780,NaN,700.0,0.789431,0.681905,0.665000,0.423188,0.355357,0.271210,0.121526,0.211523,0.266399,85.714,1.308180,0.000,0.04,1.0,NaN,0.618590,0.044898,0.025000,0.392857,315.0,0.0,NaN,1.520141e-05,0.883323,0.952825,0.000517,37.143,5009,0.5,6.390998,2.451317,1.379687,3.450677,3.792164,NaN,0.1235,False,False,False,False,False,0.428932,False,0.196557,0.090253,0.163973,0.071859,0.220139,2600,27,757988391,259,385.0,VISp,NaN,NaN,NaN,probeC,See electrode locations,29999.998448,1249.999935,True
951870245,0.000000,79.774505,0.99,0.000293,0.058202,0.000000,0.297200,850220029,60.0,0.009869,0.151853,94.97,-0.016760,0.000875,0.618032,0.946667,27.37,0.810385,490,0.686767,2.673103,4.562031,0.206030,511,55.787160,NaN,100.0,2.000000,0.690853,0.803137,5.716667,0.954167,1.344759,0.170397,0.100534,0.208702,60.000,0.148024,0.313,0.32,1.0,NaN,-0.449807,0.333333,-0.964286,-0.529412,45.0,0.0,NaN,4.347814e-05,0.041182,0.436667,0.237567,50.000,5025,0.0,0.136610,3.543751,2.586745,0.266449,0.257785,NaN,0.1055,False,False,False,False,False,0.451915,False,0.328061,0.938044,0.266418,0.637209,0.457572,2600,59,757988391,258,385.0,VISp,NaN,NaN,NaN,probeC,See electrode locations,29999.998448,1249.999935,True
951868026,0.022197,125.605885,0.99,0.000110,0.105125,0.000000,3.813412,850220029,40.0,0.067393,0.155319,477.95,-0.008496,0.000584,11.051903,1.000000,57.13,0.453266,343,2.747069,2.521571,6.468471,0.123618,364,50.405550,NaN,400.0,15.332733,1.263423,2.827253,3.042424,1.628491,0.761702,0.058936,0.028846,0.034320,90.000,5.609281,0.001,0.04,15.0,0.559233,0.407323,0.173595,0.620690,0.493506,180.0,150.0,0.037099,1.7

## Find average firing rate for static gratings. 

In [10]:
print(V1_units.firing_rate_sg)

unit_id
951867908     3.938042
951868041     3.792164
951870245     0.257785
951868026    12.981816
951868164     6.743697
951868229    15.965322
951868213    40.909139
951868375    19.738834
951868362    18.452576
951868331     4.664101
951868444    12.931857
951868431    16.513530
951868404    12.540185
951868527     0.959864
951868473     8.543525
951868633     7.590322
951868616     3.510399
951870296     3.214646
951868676     8.753350
951868744     1.017150
951868864    12.786645
951868924     7.790821
951869028     0.646127
951869012     0.740048
951869143     1.409489
951869357     0.773354
951869470    11.259255
Name: firing_rate_sg, dtype: float64


In [11]:
static_firing_rates=V1_units.firing_rate_sg 
static_average_firing_rate = sum(V1_units.firing_rate_sg)/len(V1_units.firing_rate_sg)
print(static_average_firing_rate)

8.83052009716934


## Find average firing rate for drifting gratings. 

In [12]:
print(V1_units.firing_rate_dg)

unit_id
951867908     2.648002
951868041     6.390998
951870245     0.136610
951868026     9.960849
951868164     7.207478
951868229    13.141522
951868213    30.874290
951868375    14.944451
951868362    18.125653
951868331     4.229601
951868444    10.054570
951868431     8.424786
951868404    11.098733
951868527     1.331678
951868473     8.691651
951868633     2.399139
951868616     2.254058
951870296     2.101034
951868676     7.625779
951868744     0.458542
951868864     1.034103
951868924     2.953520
951869028     0.358468
951869012     0.312402
951869143     0.207032
951869357     0.509903
951869470     6.894547
Name: firing_rate_dg, dtype: float64


In [13]:
drifting_firing_rates=V1_units.firing_rate_dg 
drifting_average_firing_rate = sum(V1_units.firing_rate_dg)/len(V1_units.firing_rate_dg)
print(drifting_average_firing_rate)

6.458125836705658


## Remaining code: I was experimenting with another way to find the average firing rate. 

In [ ]:
presentation_table = session.stimulus_presentations[session.stimulus_presentations.stimulus_name == 'natural_scenes']


presentation_times = presentation_table.start_time.values
presentation_ids = presentation_table.index.values
presentation_table.head()

In [ ]:
units_of_interest = V1_units.index.values

In [ ]:
times = session.presentationwise_spike_times(
    stimulus_presentation_ids=presentation_ids,
    unit_ids=units_of_interest
)

times.head()

In [ ]:
average_firing_rate = '.0023476'

In [ ]:
length_of_time=times.time_since_stimulus_presentation_onset

In [ ]:
print(length_of_time.index.values[4])